## Load protobuf models
To build the protobuf files, I ran:
```
cd src;
protoc --python_out=. --proto_path=protobuf protobuf/sample.proto
```

In [1]:
import sys
from google.protobuf.json_format import MessageToJson
import tempfile

sys.path.append("..")
sys.path.append("../src")
sys.path.append("../protobuf")

from ipfs import Ipfs
from sample_pb2 import Example, Type

## Create an example data record

In [2]:
example = Example(
    type = Type.FIZZ,
    content = "hello world!"
)

In [3]:
# Note: if the value of id_field_type is set to 1 (default value) it will not be included in the payload
MessageToJson(example, including_default_value_fields=True)

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

## Serialize the example record to prepare to store it in ipfs

In [4]:
content = example.SerializeToString()
content

b'\x12\x0chello world!'

## Try to deserialize it

In [5]:
reader = Example()
reader.ParseFromString(content)
MessageToJson(reader, including_default_value_fields=True)

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

## Write the serialize message to ipfs

In [6]:
ipfs = Ipfs()
filename = "jupyter_test"
ipfs.add(filename, content)

## Read the data back from ipfs

In [7]:
result = ipfs.list_files()
result

{'Entries': [{'Name': 'Tjupyter_test', 'Type': 0, 'Size': 0, 'Hash': ''},
  {'Name': 'identity', 'Type': 0, 'Size': 0, 'Hash': ''},
  {'Name': 'jupyter_test', 'Type': 0, 'Size': 0, 'Hash': ''},
  {'Name': 'loan', 'Type': 0, 'Size': 0, 'Hash': ''},
  {'Name': 'test_directory_2', 'Type': 0, 'Size': 0, 'Hash': ''},
  {'Name': 'var', 'Type': 0, 'Size': 0, 'Hash': ''}]}

In [8]:
result = ipfs.read("jupyter_test")
result

b'\x12\x0chello world!'

In [9]:
reader2 = Example()
reader2.ParseFromString(result)
result = MessageToJson(reader2, including_default_value_fields=True)
result

'{\n  "content": "hello world!",\n  "type": "FIZZ"\n}'

In [10]:
reader2.type

0

In [11]:
reader2.content

'hello world!'

## Overwrite the file with new contents

In [12]:
reader = Example()
reader.ParseFromString(ipfs.read(filename))
reader

content: "hello world!"

In [13]:
# from dataclasses import dataclass
# import subprocess
# from typing import List
# from google.protobuf.message import Message
# import os
# import requests
# import json
# from multicodec import add_prefix, remove_prefix, get_codec


# IPFS_HOME =  "/data"

# @dataclass
# class Ipfs():
    
    

#     def __init__(self, host: str = "http://127.0.0.1", port: int = 5001, version: str = "v0"):
#         self.host = host
#         self.port = port
#         self.version = version

#     def _make_request(self, method: str, endpoint: str, params: dict = None, data: dict = None, raise_for_status: bool = True):
#         url = f"{self.host}:{self.port}/api/{self.version}/{endpoint}"
#         response = requests.request(method, url, params = params, files = data)
#         if raise_for_status:
#             response.raise_for_status()
#         return response.content

#     def _dag_put(self, data: bytes) -> str:
#         try:
#             response = self._make_request(
#                 method = "POST",
#                 endpoint = "dag/put",
#                 params = {
#                     "store-codec": "raw",
#                     "input-codec": "raw"
#                 },
#                 data = {
#                     "object data": add_prefix('raw', data)
#                 },
#                 raise_for_status = False
#             )
#             print(response)
#             result = json.loads(response)
#             return result["Cid"]["/"]
#         except requests.exceptions.HTTPError as e:
#             raise RuntimeError(e.response._content.decode()) from e

#     def write(self, filename: str, data: bytes) -> None:
#         """
#         Update an existing file

#         Args:
#             filename (str): The file to update
#             data (Message): The data to overwrite the file with
#         """
#         try:
#             cid = self._dag_put(data)
#             self._make_request(
#                 method = "POST",
#                 endpoint = "files/write",
#                 params = {
#                     "arg": f"{IPFS_HOME}/{filename}",
#                     "raw-leaves": True
#                 },
#                 data = {
#                     "file": cid.encode()
#                 }
#             )
#         except requests.exceptions.HTTPError as e:
#             raise RuntimeError(e.response._content.decode()) from e

        

In [14]:
# ipfs = Ipfs()

In [15]:
new_content = Example(
    type = Type.BUZZ,
    content = "hello world???"
).SerializeToString()
new_content

b'\x08\x01\x12\x0ehello world???'

In [16]:
# Not working
# ipfs.write(filename, new_content)

In [17]:
reader = Example()
reader.ParseFromString(ipfs.read(filename))
reader

content: "hello world!"

## Delete the test file

In [18]:
ipfs.does_file_exist("jupyter_test")

True

In [19]:
ipfs.delete("jupyter_test")

In [20]:
ipfs.does_file_exist("jupyter_test")

False